In [98]:
import pandas as pd
from datetime import date,timedelta
import yfinance as yf
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
from time import sleep
from json import dumps
import json

### Step 1: Data Extraction

- get tickers list

In [99]:
# There are 2 tables on the Wikipedia page
# we want the first table

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = payload[0]
second_table = payload[1]

df = first_table

- write to file and store in s3 with API s3fs of dataframe

In [97]:
# from constant import *
# df.to_csv(
#     f"s3://{S3_BUCKET_NAME}/{S3_OUTPUT_DIRECTORY}/historical_stock_data.json",
#     index=False,
#     storage_options={"key": AWS_ACCESS_KEY, "secret": AWS_SECRET_KEY},
# )

In [100]:
tickers = df[['Symbol','Security','GICS Sector']]


In [101]:
tickers.head()

,Symbol,Security,GICS Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology


### Step 2: Transformation

In [47]:
tickers_new = tickers[:20]

In [105]:
yesterday =  str(date.today() + timedelta(days = -1))
today =  str(date.today())
# dataframe for all the tickers
stock_data = pd.DataFrame()
for index, ticker in tickers.iterrows():
    # download data
    data = yf.download(tickers = ticker['Symbol'], start = yesterday, end = today)
    # add a column for ticker
    data = data.assign(stock_index = ticker['Symbol'])
    # data = data.assign(name = ticker['Security'])
    # data = data.assign(sector = ticker['GICS Sector'])
    # add date column
    data = data.assign(date = data.index)
    # append this ticker data to larger dataframe
    stock_data = pd.concat([stock_data, data])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******


1 Failed download:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-07-08 -> 2023-07-09)')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

- convert to json format

In [106]:
# stock_data.to_csv('daily_stock_data.csv')

In [107]:
# stock_data.head()

,Open,High,Low,Close,Adj Close,Volume,stock_index,date
Date,,,,,,,,
2023-07-07,96.190002,98.989998,96.169998,97.290001,97.290001,3980411.0,MMM,2023-07-07 00:00:00
2023-07-07,70.419998,71.875000,70.410004,70.639999,70.639999,1103829.0,AOS,2023-07-07 00:00:00
2023-07-07,106.300003,107.932404,106.070000,106.349998,106.349998,4603917.0,ABT,2023-07-07 00:00:00
2023-07-07,137.149994,137.649994,135.345001,135.500000,135.500000,6627253.0,ABBV,2023-07-07 00:00:00
2023-07-07,306.730011,311.260010,305.279999,305.760010,305.760010,1888398.0,ACN,2023-07-07 00:00:00


In [52]:
stock_data123 = stock_data.to_json(orient = 'records')

### Step 3: Create Kafka Topic, Producer and Consumer

- create topic

In [40]:
admin_client = KafkaAdminClient(
    bootstrap_servers="13.250.43.64:9092", 
    client_id='client_1'
)
topic = NewTopic(name="stock_market_analysis", num_partitions=1, replication_factor=1)
admin_client.create_topics(new_topics=[topic], validate_only=False)

CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='stock_market_analysis', error_code=0, error_message=None)])

-  create producer

In [41]:
producer = KafkaProducer(
                bootstrap_servers=['54.251.68.49:9092'],
                value_serializer=lambda x: 
                dumps(x).encode('utf-8')
)

- send messsage

In [53]:
producer.send('stock_market_analysis', value = stock_data)

### Step 4: Load historical Data

In [88]:
stock_data = pd.DataFrame()
for index, ticker in tickers.iterrows():
    data = yf.download(tickers = ticker['Symbol'], start = '2023-5-31' , end = '2023-7-03')
    data = data.assign(stock_index = ticker['Symbol'])
    data = data.assign(name = ticker['Security'])
    data = data.assign(sector = ticker['GICS Sector'])
    data = data.assign(date = data.index)
    stock_data = pd.concat([stock_data, data])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******


1 Failed download:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-5-31 -> 2023-7-03)')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

In [71]:
stock_data.to_json(rf'stock_data.json', orient = 'records')

In [80]:
import pandas as pd
df = pd.read_json('historical_stock_data.json')

In [81]:
type(df)

pandas.core.frame.DataFrame

In [82]:
df.to_json(orient = 'records')

'[{"Open":121.5199966431,"High":122.6399993896,"Low":120.3700027466,"Close":122.4700012207,"Adj Close":119.0598602295,"Volume":2612800,"stock_index":"MMM","name":"3M","sector":"Industrials","date":1672704000000},{"Open":123.3499984741,"High":125.2900009155,"Low":122.7099990845,"Close":125.1500015259,"Adj Close":121.6652297974,"Volume":2769700,"stock_index":"MMM","name":"3M","sector":"Industrials","date":1672790400000},{"Open":124.2099990845,"High":124.5699996948,"Low":122.4599990845,"Close":122.9599990845,"Adj Close":119.5362091064,"Volume":2606600,"stock_index":"MMM","name":"3M","sector":"Industrials","date":1672876800000},{"Open":124.6600036621,"High":127.1299972534,"Low":123.75,"Close":126.7200012207,"Adj Close":123.1915130615,"Volume":2417000,"stock_index":"MMM","name":"3M","sector":"Industrials","date":1672963200000},{"Open":127.0,"High":129.4600067139,"Low":126.1100006104,"Close":126.7900009155,"Adj Close":123.2595596313,"Volume":2871300,"stock_index":"MMM","name":"3M","sector":"